In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import random
import copy

import sys
sys.path.append("Fall 2024")

import kmax as kx
import utility as util
from hierarchy import *
import z_solver as zs
import discretize as disc

# Continuous Edge Weights - Transition Probabilities

## Function Definitions

## Simulation

In [2]:
num_samples = 1000
lam = 0.5
offspring_dist = lambda: np.random.poisson(lam)
edge_dist = lambda: np.random.uniform(0,1)

In [3]:
# Setup transition count matrix
num_sections = 10
transitions = []

In [4]:
for _ in range(num_samples):
    # Generate tree
    tree = nx.Graph(GWBP(dist=offspring_dist, MAXLEVEL=10))
    for u,v in tree.edges():
        tree[u][v]['weight'] = edge_dist()

    if tree.number_of_nodes() == 1:
        #transition_counts[0][0] += 1
        continue
    
    # Get Z values, children
    tups = util.path_cover(tree, root=0, return_tuples=True)
    Z = {v:tup[1] for v,tup in tups}
    v1 = {v:tup[2] for v,tup in tups}
    v2 = {v:tup[3] for v,tup in tups}

    current_node = v1[0] # first max child of root 0
    while True:
        if v1[current_node] is not None:
            transitions.append((Z[current_node], Z[v1[current_node]]))
            if Z[v1[current_node]] < 0:
                break
            else:
                current_node = v1[current_node]
        else:
            # TODO - should we distinguish this case from transitions to negative?
            transitions.append((Z[current_node], -1)) 
            break

Each component `transition_counts[i][j]` represents the number of times an edge $(v_1,v_2)$ has $Z(v_1)$ in subsection $i$ and $Z(v_2)$ in subsection $j$.
That is, the row index is the source state and the column index is the end state.

In [8]:
transitions[:5]

[(0.10774726517723221, 0.4347424957848549),
 (0.4347424957848549, 0.25704362105464273),
 (0.25704362105464273, -1),
 (0.8928027309591353, -1),
 (0.6861725445215856, 0.7475370351115612)]

### TODO - fix probability conversion

In [17]:
discretized = disc.discretize_log_2d(transitions, 10).statistic
total = sum(sum(discretized))
transition_probs = discretized / total

In [18]:
transition_probs

array([[0.       , 0.       , 0.       , 0.       , 0.0015625, 0.003125 ,
        0.00625  , 0.0046875, 0.       , 0.       ],
       [0.0515625, 0.0046875, 0.0015625, 0.0015625, 0.0046875, 0.0046875,
        0.0140625, 0.003125 , 0.       , 0.       ],
       [0.0609375, 0.0015625, 0.0046875, 0.00625  , 0.00625  , 0.0078125,
        0.0171875, 0.0015625, 0.       , 0.       ],
       [0.06875  , 0.00625  , 0.00625  , 0.003125 , 0.0078125, 0.00625  ,
        0.0078125, 0.003125 , 0.       , 0.       ],
       [0.1078125, 0.003125 , 0.0046875, 0.00625  , 0.0109375, 0.0109375,
        0.0140625, 0.003125 , 0.       , 0.       ],
       [0.0953125, 0.0046875, 0.0140625, 0.003125 , 0.0109375, 0.009375 ,
        0.009375 , 0.0046875, 0.       , 0.       ],
       [0.1578125, 0.0140625, 0.0140625, 0.0171875, 0.0125   , 0.0203125,
        0.025    , 0.0078125, 0.       , 0.       ],
       [0.0671875, 0.0046875, 0.0015625, 0.00625  , 0.003125 , 0.00625  ,
        0.00625  , 0.00625  , 0.     

Rows sum to 1:

### 3d Plot

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('auto', anchor='C')

#coord matrices
x_len, y_len = transition_probs.shape

bar_scale = 0.5
dx = dy = bar_scale * (1 / num_sections)
dz = transition_probs.T.flatten()

bar_positions = section_bounds - (1 / (2 * num_sections)) - (1/4) * dx
xpos, ypos = np.meshgrid(bar_positions, bar_positions)
xpos = xpos.flatten()
ypos = ypos.flatten()
zpos = np.zeros_like(xpos)

ax.bar3d(xpos, ypos, zpos, dx, dy, dz)

ax.set_xlabel('From State')
ax.set_ylabel('To State')
ax.set_zlabel('Probability')
ax.set_title('Transition Probabilities')

ax.set_xticks(np.concatenate(([-(1 / num_sections)], section_bounds)))
ax.set_yticks(np.concatenate(([-(1 / num_sections)], section_bounds)))

ax.view_init(elev=30, azim=45, roll=0)
ax.set_box_aspect(None, zoom=0.9)
plt.show()

### Heatmap

#### Color plotter from matplotlib docs

In [ ]:
# https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current Axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(range(data.shape[1]), labels=col_labels,
                  rotation=-30, ha="right", rotation_mode="anchor")
    ax.set_yticks(range(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar

def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

#### Plot

In [ ]:
data = transition_probs[1:,1:]
labels = [str(s)[:3] for s in section_bounds[1:]]
fig, ax = plt.subplots()

im, cbar = heatmap(data, labels, labels, ax=ax,
                   cmap="YlOrBr", cbarlabel="Density")
texts = annotate_heatmap(im, valfmt="{x:.2f}")

plt.title('Transition Probabilities')
plt.xlabel('To State')
plt.ylabel('From State')

fig.tight_layout()
plt.show()

In [ ]:
[sum(data[r]) for r in range(10)]

In [ ]:
[sum(data[:,c]) for c in range(10)]

## Z histograms

In [ ]:
z_vals_along_path = []
num_samples = 100000
for _ in range(num_samples):
    # Generate tree
    tree = nx.Graph(GWBP(dist=offspring_dist, MAXLEVEL=10))
    for u,v in tree.edges():
        tree[u][v]['weight'] = edge_dist()

    if tree.number_of_nodes() == 1:
        continue
    
    # Get Z values, children
    tups = util.path_cover(tree, root=0, return_tuples=True)
    Z = {v:tup[1] for v,tup in tups}
    v1 = {v:tup[2] for v,tup in tups}
    v2 = {v:tup[3] for v,tup in tups}

    current_node = 0 # first max child of root 0
    while v1[current_node] is not None and Z[v1[current_node]] >= 0:
        current_node = v1[current_node]
        z_vals_along_path.append(Z[current_node])

In [ ]:
lam = 0.8
offspring_dist = lambda: np.random.poisson(lam)
z_vals_random = []
num_samples = 100000
for _ in range(num_samples):
        # Generate tree
    tree = nx.Graph(GWBP(dist=offspring_dist, MAXLEVEL=10))
    for u,v in tree.edges():
        tree[u][v]['weight'] = edge_dist()

    if tree.number_of_nodes() < 1:
        continue

    # Get Z values, children
    tups = util.path_cover(tree, root=0, return_tuples=True)
    # need to exclude root because we are assuming it has no z-value
    Z = {v:tup[1] for v,tup in tups if v == 0}
    z_vals_random += list(Z.values())

In [ ]:
z_count_less_than_zero = sum(1 for z in z_vals_random if z <= 0)
total_z_values = len(z_vals_random)
print(z_count_less_than_zero / total_z_values)
xs, ys = zs.compute_z_pdf(lam, 1000)

In [ ]:
z_vals_random_pos = [z for z in z_vals_random if z > 0]
fig = plt.figure(figsize=(8, 6))
fig.patch.set_facecolor('xkcd:off white')
plt.plot(xs, ys, '-', label=['Expected PDF'], color='red', linewidth = 3.0)
plt.hist(z_vals_random, bins="auto", color="blue", edgecolor="black", alpha=0.6, density="True")
plt.title(f"Root Z-Value Distribution for $\\lambda$ = {lam}, {num_samples} samples", fontfamily = ['Serif'])
plt.xlabel("Z-Value")
plt.ylabel("Normalized Frequency")
plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(z_vals_along_path, bins="auto", color="blue", edgecolor="black", alpha=0.7)
plt.title("Z-Values For Nodes Along the Path Starting From the First Max of the Root")
plt.xlabel("Z-Value")
plt.ylabel("Frequency")
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
xs, ys = zs.compute_z_pdf(0.7, 10000)

In [ ]:
plt.figure()
plt.plot(xs, ys, '-', label=['CDF', 'density'])
plt.legend()